In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, auc
from scipy.sparse import csr_matrix, hstack

In [2]:
PATH = '~/data/toxic/data/'

#train = pd.read_csv(PATH + 'cleaned_train.csv')
train = pd.read_csv('/home/kai/data/wei/Toxic/data/Shiyi_training.csv').fillna('na')

test = pd.read_csv(PATH + 'cleaned_test.csv')

train_sentence = train['comment_text_cleaned']
test_sentence = test['comment_text_cleaned']

text = pd.concat([train_sentence, test_sentence])

print(train.shape)
print(test.shape)

(32450, 27)
(153164, 21)


In [3]:
# CountVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), analyzer=’word’, max_df=1.0,\
#                 min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>)

# TfidfVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None,\
#                 lowercase=True, preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None,\
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None,\
#                 vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>, norm=’l2’, use_idf=True,\
#                 smooth_idf=True, sublinear_tf=False)

# min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
# analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1),
# use_idf=True,smooth_idf=True, sublinear_tf=True, stop_words = "english"

print('getting tfidf')
phrase_vectorizer = TfidfVectorizer(min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
                                    analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1),
                                    use_idf=True,smooth_idf=True, sublinear_tf=True, stop_words = "english")
    
#     ngram_range=(1,3),
#                                     strip_accents='unicode', 
#                                     max_features=100000, 
#                                     analyzer='word',
#                                     sublinear_tf=True,
#                                     token_pattern=r'\w{1,}')
# char_vectorizer = TfidfVectorizer(ngram_range=(1,5), 
#                                   strip_accents='unicode', 
#                                   max_features=200000, 
#                                   analyzer='char', 
#                                   sublinear_tf=True)

# print('fitting char')
# char_vectorizer.fit(text.values)
print('fitting phrase')
phrase_vectorizer.fit(text.values)
print('transforming train phrase')
train_phrase = phrase_vectorizer.transform(train_sentence.values)
# print('transforming test char')
# test_char = char_vectorizer.transform(test_sentence.values)
print('transforming test phrase')
test_phrase = phrase_vectorizer.transform(test_sentence.values)

# train_tfidf = hstack((train_char, train_phrase), format='csr')
# test_tfidf = hstack((test_char, test_phrase), format='csr')

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# train_tfidf

getting tfidf
fitting phrase
transforming train phrase
transforming test phrase


In [4]:
train_tfidf = train_phrase
test_tfidf = test_phrase

In [5]:
train_tfidf.shape, test_tfidf.shape

((32450, 205689), (153164, 205689))

In [6]:
import pdb

In [7]:
def pr(y_i, y, train_features):
    p = train_features[y==y_i].sum(0)
    return (p + 1) / ((y == y_i).sum() + 1)

def get_nblogreg_model(label_cols, train_features, train, test_features):
    #pdb.set_trace()
    preds = np.zeros((test.shape[0], len(label_cols)))
    train_preds = np.zeros((train.shape[0], len(label_cols)))
    for i, j in enumerate(label_cols):
        if i == i:
            print('fit', j)
            y = train[j].values
            r = np.log(pr(1, y, train_features) / pr(0, y, train_features))
            model = LogisticRegression()
            x_nb = train_features.multiply(r).tocsr()
            # split
#             index_to_split = int(x_nb.shape[0]*0.9)
#             x_train = x_nb[:index_to_split,:]
#             y_train = y[:index_to_split]
#             x_val = x_nb[index_to_split:,:]
#             y_val = y[index_to_split:]
#             val_preds = np.zeros((x_val.shape[0],len(label_cols)))
#             print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
#             # fit
#             model.fit(x_train, y_train)
#             # validation
#             val_preds[:, i] = model.predict(x_val)
#             val_prob =  model.predict_proba(x_val)[:, 1]
#             print('accuracy is {}'.format(accuracy_score(y_val, val_preds[:, i])))
#             print('roc_auc is {}'.format(roc_auc_score(y_val, val_prob)))
            
            model.fit(x_nb, y)
            preds[:, i] = model.predict_proba(test_features.multiply(r))[:, 1]
            train_preds[:, i] = model.predict_proba(x_nb)[:, 1]
            print('{:15} auc is {}'.format(j, roc_auc_score(y, train_preds[:, i])))
    #return preds, val_preds, val_prob, y_val, index_to_split
    return preds

def save(y_test, label_cols, path, flag):
    import time
    BUILD_ID = int(time.time())
    print('BUILD_ID: {}'.format(BUILD_ID))
    submission = pd.read_csv(path + 'sample_submission.csv')
    submission[label_cols] = y_test
    submission.to_csv('/home/kai/data/shiyi/toxic/submissions/sub_nbsvm' + '_' + flag + '_' + str(BUILD_ID) + '.csv', index=False)
    
print('done')

done


In [8]:
y_test = get_nblogreg_model(label_cols, train_tfidf, train, test_tfidf)

fit toxic
toxic           auc is 0.9680740161673966
fit severe_toxic
severe_toxic    auc is 0.9858533874442547
fit obscene
obscene         auc is 0.9774442615173118
fit threat
threat          auc is 0.9966686658880914
fit insult
insult          auc is 0.9569971192776481
fit identity_hate
identity_hate   auc is 0.9934279535675743


In [9]:
save(y_test, label_cols, PATH, '1v1')

BUILD_ID: 1519160015
